# 

# Final Makeup Recommendation Notebook

Run this notebook to upload your image and get your scientifically backed, computer vision inferenced makeup recommendation!

In [7]:
import inference_pipeline_recommendation as pipeline
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path

#### Step 1: Run the cell below and upload a photo of yourself.

In [10]:
upload = widgets.FileUpload(
    accept="image/*",
    multiple=False
)

display(upload)

FileUpload(value=(), accept='image/*', description='Upload')

#### Step 2: Run the cell below to recieve your recommendations.

In [11]:
uploaded = upload.value

UPLOAD_DIR = Path("image_uploads")
UPLOAD_DIR.mkdir(parents=True, exist_ok=True)

def save_uploaded_image(upload_widget, filename="input.jpg"):
    if not upload_widget.value:
        return None

    file_info = upload_widget.value[0]
    image_path = UPLOAD_DIR / filename

    with open(image_path, "wb") as f:
        f.write(file_info["content"])

    return image_path

if uploaded:
    file_info = uploaded[0]          # first (and only) file
    image_bytes = file_info["content"]

    from PIL import Image
    import io

    img = Image.open(io.BytesIO(image_bytes))
    save_uploaded_image(upload)

    #Load and run model

    classifiers = pipeline.load_classifiers()

    image_path = "image_uploads/input.jpg" 
    predictions = pipeline.classify_features(image_path, classifiers)
    active_features = {feat for feat, val in predictions.items() if val == 1}
    recommendations = pipeline.generate_recommendations(active_features)

    # Display outputs
    image_out = widgets.Output()
    text_out = widgets.Output()
    with image_out:
        image_out.clear_output()
        display(img)
    with text_out:
        text_out.clear_output()

        print("Detected features:")
        for f in active_features:
            print("-", f)

        print("\nMakeup Recommendations:")
        for r in recommendations:
            print("•", r)

    # --- LAYOUT (THIS GOES LAST) ---
    ui = widgets.HBox(
        [image_out, text_out],
        layout=widgets.Layout(
            align_items="flex-start",
            gap="40px"
        )
    )

    display(ui)
